<a href="https://colab.research.google.com/github/RamOmri/cits4012-project2/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "remote_proj"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)

Mounted at /content/drive

Your working directory was changed to /content/drive/My Drive/remote_proj

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [ ]:
!pip install ftfy
import ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# NLP Project 2, Seek dataset Category and Job Type Classification
22274048, 

First, import all required libraries and modules for the project and load the data into pandas

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import collections
import numpy as np
import pandas as pd
import re
import os
from collections import Counter
import string
from argparse import Namespace
import torch.nn as nn
from torch.utils.data import Dataset
import random
import os
import torch.optim as optim
import json
import torch.nn.functional as F
import pandas as pd
import unicodedata
from gensim.models import KeyedVectors
import gensim
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
import seaborn as sns
from time import time
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from gensim.models import KeyedVectors
stopwords = stopwords.words('english')

In [ ]:
script_dir = os.path.dirname(os.path.realpath('__file__'))
script_dir

'/content/drive/.shortcut-targets-by-id/1SN-8It-Z6Sun4CATVH-ShCfzdUAuexhH/remote_proj'

In [ ]:
df_args = Namespace(
    unprocessed_csv  = os.path.join(script_dir, 'data/seek_australia.csv'),
    processed_csv = os.path.join(script_dir, 'data/processed_seek.csv'),
    train_proportion=0.7,
    val_proportion=0.1,
    test_proportion=0.2,
    frac = 1, # change to 1 to use the full dataset
)

In [ ]:
#import ftfy
df = pd.read_csv (df_args.unprocessed_csv)
df = df.sample(frac=df_args.frac, ignore_index=True, random_state=1) # this also shuffles the dataframe
df

,category,city,company_name,geo,job_board,job_description,job_title,job_type,post_date,salary_offered,state,url
0,Trades & Services,"Newcastle, Maitland & Hunter",The Local Bloke Lawn & Garden Care,AU,seek,Newcastle Location Car License Required Previ...,Lawn & Garden Care Position,Casual/Vacation,2018-04-16T01:15:22Z,NaN,NaN,https://www.seek.com.au/job/35990983
1,"Manufacturing, Transport & Logistics",Sydney,B&J City Kitchen Pty Ltd,AU,seek,Beak & Johnston are industry leaders within t...,Machine Operator - Food Manufacturing,Full Time,2018-04-13T03:00:07Z,NaN,Parramatta & Western Suburbs,https://www.seek.com.au/job/35978566
2,Community Services & Development,Katherine & Northern Australia,YMCA of the Northern Territory,AU,seek,"The Opportunity Do you enjoy a challenge, lov...",Youth Diversion Worker,Full Time,2018-04-11T03:50:48Z,NaN,NaN,https://www.seek.com.au/job/35957875
3,Real Estate & Property,Sydney,Dexus,AU,seek,Work on one of the best property portfolios i...,Communications and Investor Development Manager,Full Time,2018-04-15T23:10:56Z,NaN,"CBD, Inner West & Eastern Suburbs",https://www.seek.com.au/job/35989342
4,Hospitality & Tourism,Cairns & Far North,Hays Trades & Labour,AU,seek,Permanent position on offer for a Junior Sous...,Junior Sous Chef,Full Time,2018-04-12T02:07:23Z,NaN,NaN,https://www.seek.com.au/job/35967182
...,...,...,...,...,...,...,...,...,...,...,...,...
29995,Information & Communication Technology,Sydney,Hays Information Technology,AU,seek,Junior .Net Developer required for a start-up...,Junior .Net Developer,Contract/Temp,2018-04-13T06:19:44Z,NaN,"CBD, Inner West & Eastern Suburbs",https://www.seek.com.au/job/35982781
29996,Trades & Services,Melbourne,Zoom Recruitment & Training,AU,seek,We are seeking general labourers/ hands for v...,General Labourers,Contract/Temp,2018-04-16T06:42:38Z,NaN,Bayside & South Eastern Suburbs,https://www.seek.com.au/job/35997078
29997,Sales,Melbourne,Recruitmore VIC,AU,seek,No industry experience? No worries! Use your ...,IMMEDIATE STARTS - SALES & CUSTOMER SERVICE,Contract/Temp,2018-04-14T23:00:07Z,NaN,CBD & Inner Suburbs,https://www.seek.com.au/job/35987014
29998,Construction,Sydney,Hays Construction,AU,seek,This large building construction firm are loo...,Senior Contract Administrator,Full Time,2018-04-16T02:44:21Z,NaN,"CBD, Inner West & Eastern Suburbs",https://www.seek.com.au/job/35992778


## Data preprocessing

#### Preparing the dataframe for classification

Remove unnecessary columns

In [ ]:
features_to_delete = ['job_board','geo','url', 'city', 'post_date', 'salary_offered', 'state', "company_name", "job_title"] # get rid of unnecessary features (maybe add more columns)
df = df.drop(features_to_delete, axis=1)

Retrieve the number of null values in each column

In [ ]:
df.isna().sum()

category             0
job_description    345
job_type             0
dtype: int64

Job description is the feature columns and will be used for all the classification tasks, thus all rows that are missing values for the column can be dropped

In [ ]:
df.dropna(subset=['job_description'], inplace=True)

Decode the text in the job description column and use gensim for tokenization

In [ ]:
for i in tqdm(df[ "job_description"].index):
    df.loc[i, "job_description"] = gensim.utils.simple_preprocess(
        ftfy.fix_text(unicodedata.normalize("NFKD",df.loc[i,"job_description"])), deacc=True)

  0%|          | 0/29655 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

Create_tfidf function creates a dataframe with each row representing a document with corresponding tfidf scores and the columns showing the full corpus vocabulary. Thus the tfidf score for each word in each document can be looked up.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Allows passing tokens to the vectorizer
def dummy_fun(doc):
    return doc

def create_tfidf_df(tokens):  
    """
    Returns a bag of words data frame to fetch the tfidf score for each document
    tokens: the corpus as a list of tokens
    """
    from nltk.corpus import stopwords
    stopwords = stopwords.words('english')
    tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None,
    stop_words=stopwords) 

    tdf = tfidf.fit_transform(tokens)
    bow = pd.DataFrame(tdf.toarray(), columns = tfidf.get_feature_names_out())

    return bow

Top x tfidf accepts a list of tokenized documents and removes all words that do not have the top x tfidf scores. Removes all words that do not have the top 10 tfidf scores. Treats each category as its own corpus

In [ ]:
df['short_description'] = [None for i in range(len(df))]
def top_x_tfidf(df, x):
    """
    df is the dataframe to modify, x represents the top x words to leave in each document. Returns a list of documents as tokens. 
    """
    for category in tqdm(df.category.unique()):
        tfidf_df = create_tfidf_df(df[df.category == category]['job_description'])
        col_names= set(tfidf_df.columns)
        for i, (idx_row) in enumerate(df[df.category == category].iterrows()):
            idx, row = idx_row
            desc = row['job_description']
            words = set(desc)        
            if len(words) < x: 
                df.loc[idx]['short_description'] = words
                continue
            words = words.intersection(col_names)

            # Create a dictionary of words and their tfidf values and then sort them in descending order
            tfidf_words = {word: tfidf_df.iloc[i][word] for word in words}
            tfidf_words = {k: v for k, v in sorted(tfidf_words.items(), key=lambda item: -item[1])}

            for word in desc:
                if word not in list(tfidf_words.keys())[:x]: # Remove all words who are not in the top ten tfidf scores
                    desc = list(filter(lambda x: x != word, desc))
            df.loc[idx]['short_description'] = desc
    return df['short_description']
df['short_description'] = top_x_tfidf(df, 10)
df['short_description']

In [ ]:
df['short_description']

Prepare the job type column for binary classification

In [ ]:
df.rename(columns = {'job_type': 'job_type_target'}, inplace=True) # Specify the target column
df.loc[df['job_type_target'] != 'Full Time', 'job_type_target'] = 'Other'

Observe the distribution of classes in the target column

In [ ]:
df['job_type_target'].hist()

In [ ]:
df['job_type_target'].value_counts()

Now we can define the baseline accuracy for the models as below: 

In [ ]:
1970/len(df)

Now split the data frame into train, test and validation then write the dataframe into a csv.

In [ ]:
n = len(df)
n_train = int(df_args.train_proportion*n)
n_val = int(df_args.val_proportion*n)
n_test = int(df_args.test_proportion*n)

df['split'] = ['nil' for i in range(len(df))]

for idx, _ in df[:n_train].iterrows():
    df.loc[idx]['split'] = 'train'
for idx, _ in df[n_train:n_train+n_val].iterrows():
    df.loc[idx]['split'] = 'val'
for idx, _ in df[n_train+n_val:].iterrows():
    df.loc[idx]['split'] = 'test' 
df['split'].value_counts()

In [ ]:
df.job_description = df.job_description.apply(lambda x: ' '.join(x))
df.short_description = df.short_description.apply(lambda x: ' '.join(x)) # lists cannot be stored in csv
df.to_csv(df_args.processed_csv, index=False)
df

Load the dataframe if already saved

In [ ]:
df = pd.read_csv(df_args.processed_csv)
df.short_description = df.short_description.apply(lambda x: x.split(' '))
df.job_description = df.job_description.apply(lambda x: x.split(' '))
df

## Creating word vectors to be used when training the models

In [ ]:
vectorizer_args = Namespace(
    load_word2vec = True,
    word2vec_model_filepath = os.path.join(script_dir, 'model_storage/word2vec'),
)

#### Loading the pretrained glove model

In [ ]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 200
    """
    import gensim.downloader as api
    wv_pretrained = api.load("glove-wiki-gigaword-100")
    #print("Loaded vocab size %i" % len(wv_pretrained.key_to_index))
    print("The loaded object is of type %s" % str(type(wv_pretrained)))
    return wv_pretrained
glove_model = load_embedding_model()

### Word embedding visualization with tsne and word2vec

First train or load a domain specific word2vec model

In [ ]:
if vectorizer_args.load_word2vec:
    word2vec_model = Word2Vec.load(vectorizer_args.word2vec_model_filepath).wv # Model may not load on different OS so change vector_args

else:
    # Configuration was taken from labs
    word2vec = Word2Vec(min_count=1,
                        window=2, # We are trying to find "Full time"
                        size=100,
                        sample=6e-5, 
                        alpha=0.03, 
                        min_alpha=0.0007, 
                        negative=20,
                        workers=15)
    t = time()

    word2vec.build_vocab(df['job_description'], progress_per=10)

    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    t = time()

    word2vec.train(df['job_description'], total_examples=word2vec.corpus_count, epochs=10, report_delay=1)
    word2vec_model = word2vec.wv
    word2vec_model.wv.save(vectorizer_args.word2vec_model_filepath)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Define some functions to help with plotting the words

In [ ]:
def interactive_tsne(text_labels, tsne_array):
    '''makes an interactive scatter plot with text labels for each point'''

    # Define a dataframe to be used by bokeh context
    bokeh_df = pd.DataFrame(tsne_array, text_labels, columns=['x','y'])
    bokeh_df['text_labels'] = bokeh_df.index

    # interactive controls to include to the plot
    TOOLS="hover, zoom_in, zoom_out, box_zoom, undo, redo, reset, box_select"

    p = figure(tools=TOOLS, plot_width=700, plot_height=700)

    # define data source for the plot
    source = ColumnDataSource(bokeh_df)

    # scatter plot
    p.scatter('x', 'y', source=source, fill_alpha=0.6,
              fill_color="#8724B5",
              line_color=None)

    # text labels
    labels = LabelSet(x='x', y='y', text='text_labels', y_offset=8,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')

    p.add_layout(labels)

    # show plot inline
    output_notebook()
    show(p)

In [ ]:
def display_closestwords_tsnescatterplot(model, word, vec_length):
    
    arr = np.empty((0,vec_length), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.similar_by_word(word)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.title(f'Words closest to: {word}')
    plt.show()

Below plot shows the distribution of some example words of the domain specific word2vec that was trained previously

In [ ]:
vocab  = word2vec_model.index2word[:100]
random.shuffle(vocab)
vocab = [word for word in vocab if word not in stopwords]
X = word2vec_model[vocab]
tsne = TSNE(n_components=2, random_state=0)
X_tsne = tsne.fit_transform(X)
interactive_tsne(list(vocab), X_tsne)

Now plot the same words but on the pretrained glove model

In [ ]:
vocab = [word for word in vocab if word in glove_model.index2word]
X = glove_model[vocab]
tsne = TSNE(n_components=2, random_state=0)
X_tsne = tsne.fit_transform(X)
interactive_tsne(list(vocab), X_tsne)

#### Closest words comparison

A better indication for the differences between the two embedding models would be a plot of a common word with their closest neighbours

In [ ]:
display_closestwords_tsnescatterplot(glove_model, 'fifo', 100)

In [ ]:
display_closestwords_tsnescatterplot(word2vec_model.wv, 'fifo', 100)

The above plots better communicate how the closest words are highly dependent on the source of the dataset that the embeddings were derived from. For example perth is a close word to fifo in the word2vec model but not in the pretrained glove model.

## Implementing some helper classes

The below clases are used in the implementaiton of the neural networks in the next section. 

In [ ]:
class Vocabulary(object):
    """Class to process text and extract Vocabulary for mapping"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existingmap of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                                for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)
    def add_token(self, token):
        """Update mapping dicts based on the token.
        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]
    
    def lookup_token(self, token):
        """Retrieve the index associated with the token
        or the UNK index if token isn't present.
        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
            for the UNK functionality
        """
        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    def lookup_index(self, index):
        """Return the token associated with the index
        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
        KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)

In [ ]:
class One_hot_Vectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, description_vocab, target_vocab):
        """
        Args:
            description_vocab (Vocabulary): maps words to integers
            target_vocab (Vocabulary): maps class labels to integers
        """
        self.description_vocab = description_vocab
        self.target_vocab = target_vocab
    def vectorize(self, description_tokenized):
        """Create a collapsed one hot vector for the job description
        Args:
            description_tokenized (list): the tokenized job description
        Returns:
            one_hot (np.ndarray): the collapsed onehot encoding
        """
        one_hot = np.zeros(len(self.description_vocab), dtype=np.float32)
        for token in description_tokenized:
            if token not in string.punctuation:
                one_hot[self.description_vocab.lookup_token(token)] = 1
        return one_hot
    @classmethod
    def from_dataframe(cls, description_df, target_cat = True, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            description_df (pandas.DataFrame): the description dataset
            cutoff (int): the parameter for frequency based filtering
        Returns:
            an instance of the descriptionVectorizer
        """
        description_vocab = Vocabulary(add_unk=True)
        target_vocab = Vocabulary(add_unk=False)
        # adding category or Job_type
        if target_cat:
            for category in sorted(set(description_df.category)):
                target_vocab.add_token(category)
        
        else:
            
            for job_type in  sorted(set(description_df.job_type_target)):
                target_vocab.add_token(job_type)
        # Add top words if count > provided count
        word_counts = Counter()
        for description in description_df.job_description:
            for word in description:
                if word not in string.punctuation:
                    word_counts[word] += 1
        for word, count in word_counts.items():
            if count > cutoff:
                description_vocab.add_token(word)
        return cls(description_vocab, target_vocab)
    @classmethod
    def from_serializable(cls, contents):
        """Intantiate a descriptionVectorizer from a serializable dictionary
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the descriptionVectorizer class
        """
        description_vocab = Vocabulary.from_serializable(contents['description_vocab'])
        target_vocab = Vocabulary.from_serializable(contents['target_vocab'])
        return cls(description_vocab=description_vocab, target_vocab=target_vocab)
    def to_serializable(self):
        """Create the serializable dictionary for caching
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'description_vocab': self.description_vocab.to_serializable(),
                'target_vocab': self.target_vocab.to_serializable()}

In [ ]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

In [ ]:
class Vectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""    
    def __init__(self, desc_vocab, target_vocab):
        self.description_vocab = desc_vocab
        self.target_vocab = target_vocab
    def vectorize(self, description, vector_length=-1):
        """
        Args:
            description (list) : tokenized description 
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vetorized description (numpy.array)
        """
        indices = []
        
        
        indices.extend(self.description_vocab.lookup_token(token) 
                       for token in description)
        if vector_length < 0:
            vector_length = len(indices)
        
        out_vector = np.zeros(vector_length, dtype=np.int64)
     
        out_vector[:len(indices)] = indices
       

        return out_vector
      
    def vectorize_one_hot(self, description):
        """Create a collapsed one hot vector for the job description
        Args:
            description_tokenized (list): the tokenized job description
        Returns:
            one_hot (np.ndarray): the collapsed onehot encoding
        """
        one_hot = np.zeros(len(self.description_vocab), dtype=np.float32)
        for token in description:
            if token not in string.punctuation:
                one_hot[self.description_vocab.lookup_token(token)] = 1
        return one_hot

    @classmethod
    def from_dataframe(cls, df, feature_column = 'short_description', target_cat = False, cutoff=10):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            description_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary 
        Returns:
            an instance of the descriptionVectorizer
        """
        target_vocab = Vocabulary(add_unk=False)
        if target_cat:
            for target in sorted(set(df.category)):
                target_vocab.add_token(target)    
        else:
            for job_type in sorted(set(df.job_type_target)):
                target_vocab.add_token(job_type)
        
 
        word_counts = Counter()
        for desc in df[feature_column]:
            desc = desc.split(' ') if type(desc) is not list else desc
            for token in desc:
                word_counts[token] += 1

        desc_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                desc_vocab.add_token(word)
        
        return cls(desc_vocab, target_vocab)

    @classmethod
    def from_serializable(cls, contents):
        description_vocab = \
            SequenceVocabulary.from_serializable(contents['description_vocab'])
        target_vocab =  \
            Vocabulary.from_serializable(contents['target_vocab'])

        return cls(description_vocab=description_vocab, target_vocab=target_vocab)

    def to_serializable(self):
        return {'description_vocab': self.description_vocab.to_serializable(),
                'target_vocab': self.target_vocab.to_serializable()}

In [ ]:
class Dataset(Dataset):
    def __init__(self, df, vectorizer, short = True, job_type = True, one_hot= False):
        """
        Args:
            df (pandas.DataFrame): the dataset
            vectorizer (GloveVectorizer): vectorizer instatiated from dataset
        """
        self.one_hot = one_hot
        self.df = df
        self._vectorizer = vectorizer
        self.short = short
        self.job_type = job_type

        # +1 if only using begin_seq, +2 if using both begin and end seq tokens
        measure_len = lambda context: len(context)
        if self.short:
            self._max_seq_length = max(map(measure_len, df.short_description)) + 2
        else:
            self._max_seq_length = max(map(measure_len, df.job_description)) + 2
        
        self.train_df = self.df[self.df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.df[self.df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.df[self.df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

        # Class weights
        if self.job_type:
            class_counts = df.job_type_target.value_counts().to_dict()
        else:
            class_counts = df.category.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.target_vocab.lookup_token(item[0])
        

        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)
        
        
    @classmethod
    def load_df_and_make_vectorizer(cls, df, short = True, job_type = True):
        """Load dataset and make a  vectorizer from scratch
        
        Args:
            csv (str): location of the dataset
        Returns:
            an instance of Dataset
        """
        train_df = df[df.split=='train']
        return cls(df, Vectorizer.from_dataframe(train_df), short = short)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of GloveVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return Vectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        if self.one_hot:
            if self.short:
                description_vector = \
                    self._vectorizer.vectorize_one_hot(row.short_description)
            else:
                description_vector = \
                    self._vectorizer.vectorize_one_hot(row.job_description)

        else:
            if self.short:
                description_vector = \
                    self._vectorizer.vectorize(row.short_description, self._max_seq_length)
            else:
                description_vector = \
                    self._vectorizer.vectorize(row.job_description, self._max_seq_length)

        if self.job_type:
            target_index = \
                self._vectorizer.target_vocab.lookup_token(row.job_type_target)
        else:
            target_index = \
                self._vectorizer.target_vocab.lookup_token(row.category)


        return {'x_data': description_vector,
                'y_target': target_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """ 
        return len(self) // batch_size

## Comparing neural network models on binary and multiclass classification tasks

Define some classes to help train the classification model

### Defining some helper functions for training each model

In [ ]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 4,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy_binary(y_pred, y_target):
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long().to(cnn_args.device) #.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

The make embedding function uses a word to index dictionary for creating the embedding matrix which can be passed to the classifier

In [ ]:
def make_one_hot_matrix(one_hot_vectorizer, words):
    """
    Args: 
        one_hot_vectorizer: of class one_hot_vectorizer 
        words: List of words in the dataset
    Returns:
        A matrix of the embeddings for each word matching the word's index
    """
    embedding_size = len(words)
    final_embeddings = np.zeros((len(words), len(words)))

    for i, word in enumerate(words):
      final_embeddings[i, :] = one_hot_vectorizer.vectorize(word)
    return final_embeddings

In [ ]:
def make_embedding_matrix(embeddings, word_to_idx, words):
    """
    Create embedding matrix for a specific set of words.
    
    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataseta
    """
    embedding_size = embeddings.shape[1]
    
    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

In [ ]:
def train_model(classifier, dataset, loss_func):
  """
  args:
    classifier: the pytorch model that will be trained.
    dataset: An object from the dataset class
  returns:
    the classifier and the final trainstate
  """
  classifier = classifier.to(cnn_args.device)
  dataset.class_weights = dataset_short_desc.class_weights.to(cnn_args.device)

  optimizer = optim.Adam(classifier.parameters(), lr = cnn_args.learning_rate)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode = 'min', factor=0.5, patience=1)

  train_state = make_train_state(cnn_args)

  epoch_bar = tqdm(desc='training routine', 
                            total=cnn_args.num_epochs,
                            position=0)

  dataset.set_split('train')
  train_bar = tqdm(desc='split=train',
                            total=dataset.get_num_batches(cnn_args.batch_size), 
                            position=1, 
                            leave=True)
  dataset.set_split('val')
  val_bar = tqdm(desc='split=val',
                          total=dataset.get_num_batches(cnn_args.batch_size), 
                          position=1, 
                          leave=True)


  classifier.float() # sets classifier to receive in double precision format
  try: 
      for epoch in range(cnn_args.num_epochs):
          train_state['epoch_index'] = epoch
          dataset.set_split('train')
          batch_generator = generate_batches(dataset, batch_size=cnn_args.batch_size, device=cnn_args.device)

          running_loss = 0.0
          running_acc = 0.0
          classifier.train()

          for batch_index, batch_dict in enumerate(batch_generator):
              optimizer.zero_grad()
              y_pred = classifier(batch_dict['x_data'])
  
              loss = loss_func(y_pred.float(), batch_dict['y_target'].float())
              loss_t = loss.item()
              running_loss += (loss_t - running_loss) / (batch_index + 1)
              loss.backward()
              optimizer.step()
              acc_t = compute_accuracy_binary(y_pred, batch_dict['y_target'].to(cnn_args.device))
              running_acc += (acc_t  - running_acc)/(batch_index+1)
              #update bar
              train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                                      epoch=epoch)
              train_bar.update()

          train_state['train_loss'].append(running_loss)
          train_state['train_acc'].append(running_acc)

          dataset.set_split('val')
          batch_generator = generate_batches(dataset, batch_size=cnn_args.batch_size, device=cnn_args.device)
          running_loss = 0.
          running_acc = 0.
          classifier.eval()

          for batch_index, batch_dict in enumerate(batch_generator):
              y_pred = classifier(batch_dict['x_data'])
              loss = loss_func(y_pred.float(), batch_dict['y_target'].float())
              loss_t = loss.item()
              running_loss += (loss_t - running_loss) / (batch_index + 1)
              acc_t = compute_accuracy_binary(y_pred, batch_dict['y_target'])
              running_acc += (acc_t - running_acc) / (batch_index + 1)
              val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                                  epoch=epoch)
              val_bar.update()
          train_state['val_loss'].append(running_loss)
          train_state['val_acc'].append(running_acc)

          train_state = update_train_state(args=cnn_args, model=classifier,
                                          train_state=train_state)

          scheduler.step(train_state['val_loss'][-1])

          if train_state['stop_early']:
              break
          train_bar.n = 0
          val_bar.n = 0
          epoch_bar.update()


  except KeyboardInterrupt:
      print("Exiting loop")
  return classifier, train_state

def test_model( classifier, dataset, train_state, args,
               loss_func, BCE=False):
    dataset.set_split('test')
    batch_generator = generate_batches(dataset, 
                                       batch_size=args.batch_size, 
                                       device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # compute the output
        y_pred =  classifier(batch_dict['x_data'])

        # compute the loss
        if BCE:
            loss = loss_func(y_pred, batch_dict['y_target'].float())
        else:
            loss = loss_func(y_pred, batch_dict['y_target'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
        if BCE:
            acc_t = compute_accuracy_BCE(y_pred, batch_dict['y_target'])
        else:
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)

    train_state['test_loss'] = running_loss
    train_state['test_acc'] = running_acc
    print("Test loss: {};".format(train_state['test_loss']))
    print("Test Accuracy: {}".format(train_state['test_acc']))

### CNN for Binary classification

Here some global arguments are defined to be used while training the cnn model. The cnn architecture was borrowed from lab 9 and was repurposed for binary classification.

In [ ]:
cnn_args = Namespace(
    num_epochs=100, 
    early_stopping_criteria=5,
    learning_rate=0.001, 
    dropout_p=0.5, 
    batch_size=50, 
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
    save_dir=os.path.join(script_dir, "model_storage/cnn"),
    model_state_file="cnn_model.pth",
    embedding_size = 100,
    num_channels = 50,
    hidden_dim = 200,
)
# Check CUDA
if not torch.cuda.is_available():
    cnn_args.cuda = False
    
cnn_args.device = torch.device("cuda" if cnn_args.cuda else "cpu")
print("Using CUDA: {}".format(cnn_args.cuda))
if cnn_args.expand_filepaths_to_save_dir:

    cnn_args.model_state_file = os.path.join(cnn_args.save_dir,
                                         cnn_args.model_state_file)
    
    print("Expanded filepaths: ")
    print("\t{}".format(cnn_args.model_state_file))

Defining the CNN classifier

In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_channels, hidden_dim,
     dropout_p, pretrained_embeddings = None, padding_idx = 0):
        """
        Args:
            embedding_size (int): size of the embedding vectors
            num_embeddings (int): number of embedding vectors
            filter_width (int): width of the convolutional kernels
            num_channels (int): number of convolutional kernels per layer
            hidden_dim (int): the size of the hidden dimension
            num_classes (int): the number of classes in classification
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings
                default is None. If provided, 
            padding_idx (int): an index representing a null position
        """
        super(CNNClassifier, self).__init__()

        if pretrained_embeddings is None:
            self.emb = nn.Embedding(embedding_dim = embedding_size, num_embeddings=num_embeddings, padding_idx=padding_idx)
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings)
            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx,
                                    _weight=pretrained_embeddings)
            
        self.convnet = nn.Sequential(nn.Conv1d(in_channels=embedding_size, 
                   out_channels=num_channels, kernel_size=3),
                   nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=3, stride=1),
                  nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=3, stride=2),
            nn.ELU())
        self._dropout_p = dropout_p
        self.fc1 = nn.Linear(num_channels, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, out_features = 1)
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, dataset._max_seq_length)
            apply_sigmoid (bool): a flag for the sigmoid activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, num_classes)
        """
        
        # embed and permute so features are channels
        x_embedded = self.emb(x_in).permute(0, 2, 1)

        features = self.convnet(x_embedded)

        # average and remove the extra dimension
        remaining_size = features.size(dim=2)
    
        features = F.avg_pool1d(features, remaining_size).squeeze(dim=2)
        features = F.dropout(features, p=self._dropout_p)
        
        # mlp classifier
        intermediate_vector = F.relu(F.dropout(self.fc1(features), p=self._dropout_p))
        prediction_vector = self.fc2(intermediate_vector).squeeze()

        if apply_sigmoid:
            prediction_vector = F.sigmoid(prediction_vector)
        return prediction_vector


### Training the CNN on the domain specific word2vec model

In order to explore the effectiveness of the three vectorization methods employed in this assignment, the cnn model will be trained using each of them when deriving the embeddings while using the short description column to train a baseline model.

#### Baseline model trained on the short descriptions

In [ ]:
# Prepare embeddings for training loop and initialize the classifier
dataset_short_desc = Dataset.load_df_and_make_vectorizer(df)
dataloader_short_desc = DataLoader(dataset=dataset_short_desc, batch_size=cnn_args.batch_size,
                            shuffle=True, drop_last=True)

vectorizer_short_desc = dataset_short_desc.get_vectorizer()
words = vectorizer_short_desc.description_vocab._token_to_idx.keys()
word_to_idx = {word: idx for idx, word in enumerate(word2vec_model.index2word)}
embeddings = make_embedding_matrix(word2vec_model.vectors, word_to_idx, words = words)

cnn_classifier_short_word2vec = CNNClassifier(embedding_size=embeddings.shape[1], 
                            num_embeddings=len(vectorizer_short_desc.description_vocab),
                            num_channels=cnn_args.num_channels,
                            hidden_dim=cnn_args.hidden_dim,  
                            dropout_p=cnn_args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)
loss_func = nn.BCEWithLogitsLoss()

# train the model 
cnn_classifier_short_word2vec, train_state = train_model(cnn_classifier_short_word2vec, dataset_short_desc, loss_func)
cnn_classifier_short_word2vec

#### Full description model with word2vec embeddings

In [ ]:
# Prepare embeddings for training loop and initialize the classifier
dataset_long_desc = Dataset.load_df_and_make_vectorizer(df, short=False)
dataloader_long_desc = DataLoader(dataset=dataset_long_desc, batch_size=cnn_args.batch_size,
                            shuffle=True, drop_last=True)

vectorizer_long_desc = dataset_long_desc.get_vectorizer()
words = vectorizer_long_desc.description_vocab._token_to_idx.keys()
word_to_idx = {word: idx for idx, word in enumerate(word2vec_model.index2word)}
embeddings = make_embedding_matrix(word2vec_model.vectors, word_to_idx, words = words)

cnn_classifier_long_word2vec = CNNClassifier(embedding_size=embeddings.shape[1], 
                            num_embeddings=len(vectorizer_long_desc.description_vocab),
                            num_channels=cnn_args.num_channels,
                            hidden_dim=cnn_args.hidden_dim,  
                            dropout_p=cnn_args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)
loss_func = nn.BCEWithLogitsLoss()

# train the model 
cnn_classifier_long_word2vec, train_state = train_model(cnn_classifier_long_word2vec, dataset_long_desc, loss_func)
cnn_classifier_long_word2vec

#### Training the CNN classifier using GLOVE embeddings

#### Baseline model with glove embeddings

In [ ]:
# Prepare embeddings for training loop and initialize the classifier
words = vectorizer_short_desc.description_vocab._token_to_idx.keys()
word_to_idx = {word: idx for idx, word in enumerate(glove_model.index2word)}
embeddings = make_embedding_matrix(glove_model.vectors, word_to_idx, words = words)

cnn_classifier_short_glove = CNNClassifier(embedding_size=embeddings.shape[1], 
                            num_embeddings=len(vectorizer_short_desc.description_vocab),
                            num_channels=cnn_args.num_channels,
                            hidden_dim=cnn_args.hidden_dim,  
                            dropout_p=cnn_args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)
loss_func = nn.BCEWithLogitsLoss()

# train the model 
cnn_classifier_short_glove, train_state = train_model(cnn_classifier_short_glove, dataset_short_desc, loss_func)
cnn_classifier_short_glove

#### Full description model with glove embeddings

In [ ]:
# Prepare embeddings for training loop and initialize the classifier
words = vectorizer_long_desc.description_vocab._token_to_idx.keys()
word_to_idx = {word: idx for idx, word in enumerate(glove_model.index2word)}
embeddings = make_embedding_matrix(glove_model.vectors, word_to_idx, words = words)

cnn_classifier_long_glove = CNNClassifier(embedding_size=embeddings.shape[1], 
                            num_embeddings=len(vectorizer_long_desc.description_vocab),
                            num_channels=cnn_args.num_channels,
                            hidden_dim=cnn_args.hidden_dim,  
                            dropout_p=cnn_args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)
loss_func = nn.BCEWithLogitsLoss()

# train the model 
cnn_classifier_long_glove, train_state = train_model(cnn_classifier_long_glove, dataset_long_desc, loss_func)
cnn_classifier_long_glove

### Training the cnn on one hot encoded dataset

#### Baseline one hot encoded classifier

In [ ]:
one_hot = One_hot_Vectorizer.from_dataframe(df, target_cat = False)
words = one_hot.description_vocab._token_to_idx.keys()
embeddings = make_one_hot_matrix(one_hot, words)

cnn_classifier_short_one_hot = CNNClassifier(embedding_size=len(words), 
                            num_embeddings=len(words),
                            num_channels=cnn_args.num_channels,
                            hidden_dim=cnn_args.hidden_dim,  
                            dropout_p=cnn_args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)
loss_func = nn.BCEWithLogitsLoss()

# train the model 
cnn_classifier_short_one_hot, train_state = train_model(cnn_classifier_short_one_hot, dataset_short_desc, loss_func)
cnn_classifier_short_one_hot

#### One hot encoding on the shortened job description

In [ ]:
one_hot = One_hot_Vectorizer.from_dataframe(df, target_cat = False)
words = one_hot.description_vocab._token_to_idx.keys()
embeddings = make_one_hot_matrix(one_hot, words)

cnn_classifier_short_one_hot = CNNClassifier(embedding_size=len(words), 
                            num_embeddings=len(words),
                            num_channels=cnn_args.num_channels,
                            hidden_dim=cnn_args.hidden_dim,  
                            dropout_p=cnn_args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)
loss_func = nn.BCEWithLogitsLoss()

# train the model 
cnn_classifier_short_one_hot, train_state = train_model(cnn_classifier_short_one_hot, dataset_short_desc, loss_func)
cnn_classifier_short_one_hot

#### One hot encoding on the full job description

In [ ]:
one_hot = One_hot_Vectorizer.from_dataframe(df, target_cat = False)
words = one_hot.description_vocab._token_to_idx.keys()
embeddings = make_one_hot_matrix(one_hot, words)

cnn_classifier_long_one_hot = CNNClassifier(embedding_size=len(words), 
                            num_embeddings=len(words),
                            num_channels=cnn_args.num_channels,
                            hidden_dim=cnn_args.hidden_dim,  
                            dropout_p=cnn_args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)
loss_func = nn.BCEWithLogitsLoss()

# train the model 
cnn_classifier_long_one_hot, train_state = train_model(cnn_classifier_long_one_hot, dataset_long_desc, loss_func)
cnn_classifier_long_one_hot

In [ ]:
#compute the loss & accuracy on the test set using the best available model

cnn_classifier.load_state_dict(torch.load(train_state['model_filename']))

dataset_long_desc.class_weights = dataset_long_desc.class_weights.to(cnn_args.device)
loss_func = nn.CrossEntropyLoss(dataset_long_desc.class_weights)

dataset_long_desc.set_split('test')
batch_generator = generate_batches(dataset_long_desc, 
                                   batch_size=cnn_args.batch_size, 
                                   device=cnn_args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy_binary(y_pred.float(), batch_dict['y_target'].float())
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))